In [17]:
import requests

with open('Q&A_format.md', 'r', encoding='utf-8') as file:
    data = file.read()

# Manually split the document based on headers
questions_answers = data.split("Question: ")

# Prepare the data for RAG
rag_data = []
for qa in questions_answers:
    if qa.strip():  # Ignore any empty strings
        question, *answer = qa.split("Answer: ")
        if answer:
            rag_data.append({
                'question': question.strip(),
                'answer': answer[0].strip()
            })

# Function to query the localhost server
def query_localhost(prompt):
    try:
        response = requests.post("http://localhost:1234/v1", json={"prompt": prompt})
        response.raise_for_status()  # Ensure we raise an error for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return {"generated_text": "Error: Unable to process the request."}

# Function to get an answer using RAG
def get_rag_answer(question):
    # Retrieve relevant document section (simulating the retrieval step)
    relevant_answer = None
    for entry in rag_data:
        if question.lower() in entry['question'].lower():
            relevant_answer = entry['answer']
            break
    
    # Generate the final answer using the model
    prompt = f"Question: {question}\n\nAnswer: {relevant_answer if relevant_answer else ''}"
    generated_answer = query_localhost(prompt)
    
    return generated_answer.get(prompt, generated_answer)

# Example usage
question = "What is the capital of France?"
answer = get_rag_answer(question)
print(answer)

{'error': 'Unexpected endpoint or method. (POST /v1)'}


In [20]:
from pathlib import Path

from langchain.llms import OpenAI
import chromadb

from langchain_openai import OpenAIEmbeddings

from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
#from langchain_community.embeddings import GPT4AllEmbeddings
from langchain import hub
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("loading model")
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

fulldir = Path.home() / 'OneDrive' / 'Documents' / 'throawaylien'
# C:\Users\joshs\OneDrive\Documents\throawaylien
#loaderTEXT = TextLoader(pathy)
dirloader = DirectoryLoader(fulldir.absolute(), glob='**/*.txt', loader_cls=TextLoader)
#loaderPDF = PyPDFLoader(pathypdf)
print("instantiated loader")
dirdata = dirloader.load()

# print("Data was: ", data)
print("splitting text and embedding using gpt4all embeddings")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
splits = text_splitter.split_documents(dirdata)

embeddings = OpenAIEmbeddings(
    base_url="http://localhost:1234/v1",
    api_key="n/a",
    model="nomic-ai/nomic-embed-text-v1.5-GGUF",
    # model="text-embedding-3-small",
    # embedding_ctx_length=1000,
    # tiktoken_enabled=True,
    )
new_client = chromadb.EphemeralClient()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
print("finished the vectorestore")
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = client

template = """Use the provided pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.

CONTEXT:

```{context}```

QUESTION: {question}

HELPFUL ANSWER:"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def enter_question():
    print("about to invoke the rag_chain")
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )

    question = input("Enter your prompt: ")
    for chunk in rag_chain.stream(question):
        print(chunk, end="", flush=True)
    print("just finished invoking the rag_chain")
    # cleanup

while True:
    enter_question()

vectorstore.delete_collection()

RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m